### Problem:
The images are in different orientations: upright 'in label:' rotated_right ',' rotated_left ',' upside_down 'and' upright '.

### Solution: 
A neural network must be trained to solve the test problem. There is a train.csv file that contains a column called label (the learning algorithm) and in the fn column the name of the file (extraction of characteristics). The images must be opened as a vector, so that NN can obtain the characteristics. In this case, supervised learning should be used due to needing to learn the difference between the positions of the images.

# Attempt 1- Image classification from keras

In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

import gc

In [2]:
from keras import layers, models, optimizers

Using TensorFlow backend.


In [3]:
from keras.preprocessing.image import (
    img_to_array,
    load_img,
    ImageDataGenerator
)

In [4]:
train_imgs = pd.read_csv('./train.csv')

In [5]:
train_imgs.head(10)

,fn,label
0,0-10049200_1891-09-16_1958.jpg,rotated_left
1,0-10110600_1985-09-17_2012.jpg,rotated_left
2,0-10126400_1964-07-07_2010.jpg,upright
3,0-1013900_1917-10-15_1960.jpg,rotated_right
4,0-10166400_1960-03-12_2008.jpg,upside_down
5,0-102100_1970-10-09_2008.jpg,rotated_left
6,0-1024100_1982-06-07_2011.jpg,rotated_left
7,0-10292500_1984-03-26_2009.jpg,rotated_left
8,0-1035700_1945-11-27_2013.jpg,upside_down
9,0-10416800_1907-01-08_1967.jpg,upright


In [6]:
#train_imgs['fn'][0:5]

In [7]:
#train_imgs['fn'][2]

View images

In [8]:
import matplotlib.image as mpimg

for ima in train_imgs['fn'][0:15]:
    img = mpimg.imread('./train/'+ima)
    #imgplot = plt.imshow(img)
    #plt.show()
    #print(img.shape)

Function to separate the characteristics of the images of the learning

In [9]:
def read_and_process_image(lista_img,lista_lab):
    
    
    X = []
    y =[]
    
    for image,label in zip(lista_img,lista_lab):
        
        X.append(mpimg.imread('./train/'+image))# Ler imagem em vetor
        
        # Change string to int (upright = 0, right = 1, left = 2, upside down =3)
        if 'upright' in label:
            y.append(0)
        elif 'rotated_right' in label:
            y.append(1)
        elif 'rotated_left' in label:
            y.append(2)
        elif 'upside_down' in label:
            y.append(3)
        else:
            y.append(None)
    return X,y

In [10]:
len(train_imgs['fn'])

48896

Get 1100 images

In [11]:
X,y = read_and_process_image(train_imgs['fn'][0:10000],train_imgs['label'][0:10000])

In [12]:
# turn into Array
X = np.array(X)
y = np.array(y)

In [13]:
del train_imgs
gc.collect()

29

### -----------------ESTUDO IMAGENS----------------

In [14]:
X.shape #VETOR GUARDADO AS IMGs
#1100 imagens... camada imagem tem 64 linhas, 64 colunas e 3 camadas (RGB)

(10000, 64, 64, 3)

In [15]:
#X[0].shape # PEGANDO UMA IMAGEM

In [16]:
#X[0] # 1 imagem

In [17]:
#X[0][0] # Linhas índice 0 ( que contem 64 colunas e 3 camadas em cada coluna)

In [18]:
X[0][0][0] # Coluna 0 que contem 3 camadas

array([126, 125, 130], dtype=uint8)

Linha (y) e coluna (x) ... ou seja a linha e coluna indica a posição...  em cada linha temos 64 colunas e em cada coluna temos 3 camadas. Essas 3 camadas tem valores de 0 até 255... se misturam para formar uma COR.

Portanto, as linhas e colunas são a posição espacial dos pontos no gráfico, e a cor é defenida pela mistura dos valores das 3 camadas RGB.. (ex: [0,0,255] = AZUL).

### ---------------FIM ESTUDOS IMAGENS ----------------

counting different positions

analyze the X and y shape

In [19]:
#4D batch size, height, width and channels
print('Shape of train imag is:', X.shape)

#1D
print('Shape of train imag is:', y.shape)

Shape of train imag is: (10000, 64, 64, 3)
Shape of train imag is: (10000,)


splitting data arrays into two subsets

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.20,
    random_state =2
)

In [22]:
del X
del y
gc.collect()

0

In [23]:
X_train.shape, y_train.shape

((8000, 64, 64, 3), (8000,))

In [24]:
ntrain = len(X_train)
nval = len(X_val)

In [25]:
batch_size =64

begin create the model

In [26]:
"""model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation = 'relu',input_shape=(64,64,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation = 'relu'))
model.add(layers.Dense(1, activation = 'relu'))"""
model = models.Sequential()
model.add(layers.Conv2D(filters = 32,kernel_size=(3,3),activation = 'relu',input_shape=(64,64,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(64,activation = 'relu'))
model.add(layers.Dense(4, activation = 'softmax'))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 128)        

In [28]:
"""""model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(learning_rate=1e-4),metrics = ['acc'])"""""
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "Adam",metrics = ['accuracy'])

In [29]:
x = 0.2

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 0,
    width_shift_range = x,
    height_shift_range= x,
    shear_range= x,
    zoom_range= x,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale = 1./255)

In [30]:
train_generator = train_datagen.flow(X_train,y_train,batch_size = batch_size)
val_generator = val_datagen.flow(X_val,y_val,batch_size = batch_size)


In [33]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = ntrain // batch_size,
    epochs = 40,
    validation_data = val_generator,
    validation_steps = nval // batch_size
)

Epoch 1/40
125/125 [==============================] - 42s 337ms/step - loss: 0.4321 - accuracy: 0.7505 - val_loss: 0.5061 - val_accuracy: 0.7107
Epoch 2/40
 67/125 [===============>..............] - ETA: 18s - loss: 0.4158 - accuracy: 0.7451

KeyboardInterrupt: 

In [ ]:
#model.save_weights('./treino/salvar_treino_01')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs,acc,'*-b',label = 'Training Accurarcy')
plt.plot(epochs,val_acc,'o-r',label = 'Validation Accurarcy')
plt.title('Training and Valdation Accurarcy')
plt.legend()

plt.figure()
plt.plot(epochs,loss,'*-b',label = 'Training loss')
plt.plot(epochs,val_loss,'o-r',label = 'Validation loss')
plt.title('Training and Valdation Accurarcy')
plt.legend()

plt.show()

In [ ]:

model = models.Sequential()
model.add(layers.Conv2D(filters = 32,kernel_size=(3,3),activation = 'relu',input_shape=(64,64,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(64,activation = 'relu'))
model.add(layers.Dense(4, activation = 'softmax'))


In [ ]:
x = 0.2

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 0,
    width_shift_range = x,
    height_shift_range= x,
    shear_range= x,
    zoom_range= x,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow(X_train,y_train,batch_size = batch_size)
val_generator = val_datagen.flow(X_val,y_val,batch_size = batch_size)


In [32]:
model.load_weights('./treino/salvar_treino_01')

In [ ]:


history = model.fit_generator(
    train_generator,
    steps_per_epoch = ntrain // batch_size,
    epochs = 100,
    validation_data = val_generator,
    validation_steps = nval // batch_size
)

As the epoch increased has a decrease in the accuracy of the data, then no need to increase the number of seasons.